### 정책 기반 에이전트 - REINFORCE 구현

In [ ]:
# DQN algorithm for CartPole 동작 과정 요약


In [12]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Hyperparameters
learning_rate = 0.0002
gamma = 0.98

In [27]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2) # 취할 수 있는 Action의 개수가 2개이므로
        self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim = 0)
        return x
    
    def put_data(self, item): # REINFORCE 알고리즘에서는 pi_theta(s_t, a_t)와 G_t만 있으면 loss를 계산할 수 있기 때문에 확률값 prob[a]와 보상 r을 데이터에 저장
        self.data.append(item)

    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R # Episode의 맨 뒤의 데이터부터 보면서 reward를 누적 (따라서 R은 G_t)
            loss = - R * torch.log(prob) # Gradient Ascent를 하기 위해서 -를 붙임 / 더 좋은 액션을 강화하는 형태
            loss.backward() # loss를 계산한 후 각 파라미터에 대해 미분값을 계산
        self.optimizer.step() # parameter 업데이트
        self.data = [] # 데이터 초기화

##### Q. 여기서 Loss, loss.backward()의 parameter는 각 (S, A) 별로 존재하는게 맞는지

In [28]:
def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20

    for n_epi in range(100): # 10000으로 수정필요
        s, _ = env.reset() # s = 길이 4 벡터
        done = False

        while not done:
            prob = pi(torch.from_numpy(s).float()) # prob : tensor([0.4491, 0.5509], grad_fn=<SoftmaxBackward0>)
            m = Categorical(prob)
            a = m.sample() # a: tensor(1) /  각 Category에 대한 확률을 계산하고, 그 중 하나를 뽑는다.(type : tensor) / a = 0 or 1
            # prob가 높은 action은 더 자주, prob가 낮은 action은 덜 뽑히게 된다.
            s_prime, r, done, truncat1ed, info = env.step(a.item()) # a.item() : tensor를 int로 바꿔준다.
            pi.put_data((r, prob[a])) # r : 1 / prob[a] : tensor(0.5509)
            s = s_prime
            score += r

        pi.train_net() # episode가 끝나면 하나의 episode동안 모은 데이터를 이용해 Parameter(theta) Update를 진행
        
        if n_epi % print_interval == 0 and n_epi != 0: # 20번의 episode마다 avg score 출력
            print("# of episode : {}, avg score : {}".format(n_epi, score / print_interval))
            score = 0.0
            
    env.close()

if __name__ == '__main__':
    main()

# of episode : 20, avg score : 24.0
# of episode : 40, avg score : 22.55
# of episode : 60, avg score : 29.35
# of episode : 80, avg score : 23.05


In [ ]:
R이 어떻게 정의되는거지?

### 정책 기반 에이전트 - TD Actor - Critic 구현